In [26]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [27]:
torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation(10),
                        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.131,), (0.308,)) # mean 0.13101533792088266
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.131,), (0.308,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [6]:
# Data to plot accuracy and loss graphs
train_losses = []
test_losses = []
train_acc = []
test_acc = []
train_batch_losses = []
train_batch_acc = []
test_batch_losses = []
test_batch_acc = []

def GetCorrectPredCount(pPrediction, pLabels):
  return pPrediction.argmax(dim=1).eq(pLabels).sum().item()

In [28]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, criterion):
    model.train()
    pbar = tqdm(train_loader)
    train_loss = 0
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        #loss = F.nll_loss(output, target)
        loss = criterion(output, target)
        train_loss+=loss.item()
        train_batch_losses.append(loss.item()) # Store batch loss
        loss.backward()
        optimizer.step()
        correct += GetCorrectPredCount(output, target)
        processed += len(data)
        Accuracy = float(100*correct/processed)
        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accuracy={Accuracy}')
        train_batch_acc.append(Accuracy) # Store batch accuracy
        pbar.set_description(desc= f'loss={train_loss/len(train_loader)} batch_id={batch_idx} Accuracy={Accuracy}')
    train_acc.append(Accuracy)
    train_losses.append(train_loss/len(train_loader))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            #test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            loss = criterion(output, target).item()
            test_loss += loss
            test_batch_losses.append(loss)
            #pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            #correct += pred.eq(target.view_as(pred)).sum().item()
            correct += GetCorrectPredCount(output, target)
            test_batch_acc.append(100. * correct / len(test_loader.dataset)) # Store batch accuracy

    test_loss /= len(test_loader.dataset)
    test_acc.append(100. * correct / len(test_loader.dataset))
    test_losses.append(test_loss)
    Accuracy = float(float(100. * correct) / len(test_loader.dataset))

    print('\nTest set: Average loss: {:.2f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        Accuracy))

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1, bias=False),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8, 16, 3, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 40, 3, padding=1, bias=False),
            nn.BatchNorm2d(40),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(40, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [30]:
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
       BatchNorm2d-2            [-1, 8, 28, 28]              16
              ReLU-3            [-1, 8, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           1,152
       BatchNorm2d-5           [-1, 16, 28, 28]              32
              ReLU-6           [-1, 16, 28, 28]               0
         MaxPool2d-7           [-1, 16, 14, 14]               0
            Conv2d-8           [-1, 32, 14, 14]           4,608
       BatchNorm2d-9           [-1, 32, 14, 14]              64
             ReLU-10           [-1, 32, 14, 14]               0
           Conv2d-11           [-1, 48, 14, 14]          13,824
      BatchNorm2d-12           [-1, 48, 14, 14]              96
             ReLU-13           [-1, 48, 14, 14]               0
        MaxPool2d-14             [-1, 4

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.2)
scheduler.state_dict()

{'step_size': 15,
 'gamma': 0.1,
 'base_lrs': [0.001],
 'last_epoch': 0,
 '_step_count': 1,
 '_is_initial': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [0.001]}

In [ ]:
num_epochs = 15
for epoch in range(1, num_epochs+1):
    print(f'Epoch {epoch}')
    train(model, device, train_loader, optimizer, criterion)
    test(model, device, test_loader)
    scheduler.step()

Epoch 1


loss=27936.302229225636 batch_id=1874 Accuracy=89.34833333333333: 100%|██████████| 1875/1875 [05:22<00:00,  5.81it/s]



Test set: Average loss: 0.15, Accuracy: 9598/10000 (95.98%)

Epoch 2


loss=7300.391744673252 batch_id=1874 Accuracy=96.705: 100%|██████████| 1875/1875 [03:03<00:00, 10.24it/s]            



Test set: Average loss: 0.48, Accuracy: 8207/10000 (82.07%)

Epoch 3


...


Test set: Average loss: 0.04, Accuracy: 9886/10000 (98.86%)

Epoch 10


loss=2117.9342103116214 batch_id=1874 Accuracy=99.00833333333334: 100%|██████████| 1875/1875 [03:30<00:00,  8.92it/s]



Test set: Average loss: 0.04, Accuracy: 9880/10000 (98.80%)

Epoch 11


loss=1534.1922248350456 batch_id=1502 Accuracy=99.01239188290087:  80%|████████  | 1503/1875 [04:41<01:09,  5.35it/s]


KeyboardInterrupt: 